In [ ]:
import numpy as np
import distance
import pandas as pd

# The Qmatrix algorithm is a simple hill-climbing algorithm that creates a matrix representing relationships between concepts and questions directly from student response data.
#@param DataFrame: questions (columns) and responses (rows)
#@param NumStarts: number of starts
#@param NumIter: number of iterations
#@param InitialCon: number of concepts to start with
#@param NumCon: number of concepts to end with
#@param NumQues: number of questions
#@param Delta: delta that is added or subtracted

# E.g. clusterQmatrix(df.round(), 5, 5, 1, 4, len(df.columns), 0.5)


def clusterQmatrix(DataFrame, NumStarts, NumIter, InitialCon, NumCon, NumQues, Delta):
    responselist = DataFrame.values.tolist()
    LargeNumber = 100000000000000
    MinError = LargeNumber
    NumStarts = range(0,NumStarts)
    NumIter = range(NumIter)
    NumQues = range(NumQues)
    # Set stopping criterion concepts
    k = NumCon 
    # Set initial concept(s)
    i = InitialCon

    while i < k:
        for starts in NumStarts:
            Q = np.random.random((i, len(NumQues)))
            Qnew = Q.copy()
            CurrError = Error(Q,i,responselist)
            for it in NumIter:
                for c in range(0, i):
                    print('start:', starts,'iteration:', it, 'concept:', c)
                    for q in NumQues:
                        Qnew[c][q] = Q[c][q].copy() + Delta
                        if (Error(Qnew,i,responselist) < CurrError):
                            while (Error(Qnew,i,responselist) < CurrError):
                                Q=Qnew.copy()
                                CurrError = Error(Qnew,i,responselist)
                                Qnew[c][q] = Q[c][q].copy() + Delta
                        else:
                            Qnew[c][q] = Q[c][q].copy() - Delta
                            while (Error(Qnew,i,responselist) < CurrError):
                                Q=Qnew.copy()
                                CurrError = Error(Qnew,i,responselist)
                                Qnew[c][q] = Q[c][q].copy() - Delta
            if (CurrError < MinError):
                BestQ = Q
                print('BestQ', BestQ)
                MinError = CurrError 
                print('Minerror',MinError)
        i += 1    


def calculateIdealResponseVector(QI, ci):
# Create concept state list
    ConStat= []
# Get concept states
    for i in range(ci * ci):
        b = bin(i)[2:]
        if(ci == 1):
            ConStat.append('0')
            ConStat.append('1')
        if(ci == (len(b.zfill(ci)))):
            ConStat.append(b.zfill(ci))
# Transpose Q-matrix
    QR = QI.T
# Create empty q-matrix list
    qrlist = []
# Get response vectors
    for qrt in QR:
        string = map(str, qrt)  
        qrlist.append(''.join(string))
# Create ideal response vectors (ideal response vector)
    IDR = []
# Get IDR
    for s in ConStat:
        idritem = []
        for q in qrlist:
            if(s >= q):
                idritem.append(1)
            else:
                idritem.append(0)
        IDR.append(idritem)
    return IDR

def Error(QE,ce,responselist):
    errorlist =[]
    for res in responselist:
        idrs = calculateIdealResponseVector(QE,ce)
        # large error number
        err=100000
        for idr in idrs:
            hammingdistance = distance.hamming(res, idr)
            if((err > hammingdistance)):
                err = hammingdistance
        errorlist.append(err)
    overallerror = float(sum(errorlist))/float(len(errorlist))
    return overallerror 








            
